In [1]:
import cx_Oracle
import os
import io

In [2]:
os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
connection = cx_Oracle.connect("C##PPAWLUK", "#Lex2018", "144.21.89.60:1521/ORCL.591263988.oraclecloud.internal")

In [3]:
cur = connection.cursor()
cur1 = connection.cursor()
cur2 = connection.cursor()
cur3 = connection.cursor()
cur4 = connection.cursor()
cur5 = connection.cursor()

In [4]:
cur.execute('DELETE FROM "C##PPAWLUK"."DOKUMENT"')
cur.execute('commit')

In [5]:
nameDocuments = os.listdir("dokumenty")

In [6]:
def clearTxt(myfile):
    myfile = myfile.replace('\n', ' ').replace(',', '').replace('.', '').replace('!', '').replace('?', '').replace('/',
                                                                                                                   '').replace(
        '\\', '').replace(';', '').replace('"', ' ').replace('\'', ' ').replace(']', '').replace('[', '').replace(')',
                                                                                                                  '').replace(
        '(',
        '').replace(
        ':', '').replace('`', '').replace('@', '').replace('#', '').replace('$', '').replace('%', '').replace(
        '^', '').replace('&', '').replace('*', '').replace('-', ' ').replace('_', ' ').replace('+', ' ').replace('=',
                                                                                                                 ' ').replace(
        '0', '').replace('1', '').replace('2', '').replace('3', '').replace('4', '').replace('5', '').replace('6',
                                                                                                              '').replace(
        '7', '').replace('8', '').replace('9', '').replace('>', ' ').replace('<', ' ').replace('{', '').replace('}',
                                                                                                                '').replace(
        '§', '').replace('\', '').replace('\\ufeff', '').replace(' ', ' ').replace('–', ' ').replace('“', '').replace(
        '„', '').replace('…', '').replace('	', '').replace('’', ' ').replace('”', '').replace('  ', ' ').replace(
        '  ', ' ')
    return myfile

In [7]:
def stopList(myfile):
    replaceFile = myfile
    for word in myfile:
        print(word)
        stopword = cur1.execute(
            'select STWORD from STOPWORDS where STWORD = \'{}\' and rownum=1'.format(
                word))
        for i in stopword:
            replaceFile.remove(i[0])
    removeTab = {'	', '\n', ''}
    for remove in removeTab:
        while (remove in replaceFile):
            replaceFile.remove(remove)
    return replaceFile

In [8]:
def slowikPolskiego(replaceFile):
    j = 0
    for word in replaceFile:
        dictionary = cur2.execute(
            'select slow_forma_podstawowa from slownik where slow_wyraz = \'{}\' and rownum=1'.format(
                word))
        for i in dictionary:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile


In [9]:
def slowikSynonimow1(replaceFile):
    j = 0
    for word in replaceFile:
        thesaurus = cur.execute(
            'select slow_forma_podstawowa from slownik_synonimow where slow_wyraz = \'{}\' and slow_wyraz != slow_forma_podstawowa and rownum=1 '.format(
                word))
        for i in thesaurus:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile

In [10]:
def slowikSynonimow2(replaceFile):
    j = 0
    for word in replaceFile:
        thesaurus = cur.execute(
            'select slowo_podstawowe from synonimy_orginalne where (synonim1 = \'{}\' or synonim2 = \'{}\' or synonim3 = \'{}\' or synonim4 = \'{}\' or synonim5 = \'{}\' or synonim6 = \'{}\' or synonim7 = \'{}\' or synonim8 = \'{}\' or synonim9 = \'{}\' or synonim10 = \'{}\' or synonim11 = \'{}\' or synonim12 = \'{}\' or synonim13 = \'{}\' or synonim14 = \'{}\' or synonim15 = \'{}\' or synonim16 = \'{}\' or synonim17 = \'{}\' or synonim18 = \'{}\') and NOT EXISTS(select slowo_podstawowe from synonimy_orginalne where slowo_podstawowe = \'{}\') and rownum=1 '.format(
                word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word))
        for i in thesaurus:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile

In [11]:
def slowikSynonimow3(replaceFile):
    j = 0
    for word in replaceFile:
        thesaurus = cur3.execute(
            'select COLUMN1 from SYNONIMY2 where (COLUMN2 = \'{}\' or COLUMN3 = \'{}\' or COLUMN4 = \'{}\' or COLUMN5 = \'{}\' or COLUMN6 = \'{}\' or COLUMN7 = \'{}\' or COLUMN8 = \'{}\' or COLUMN9 = \'{}\' or COLUMN10 = \'{}\' or COLUMN11 = \'{}\' or COLUMN12 = \'{}\' or COLUMN13 = \'{}\' or COLUMN14 = \'{}\' or COLUMN15 = \'{}\' or COLUMN16 = \'{}\' or COLUMN17 = \'{}\' or COLUMN18 = \'{}\' or COLUMN19 = \'{}\' or COLUMN20 = \'{}\' or COLUMN21 = \'{}\' or COLUMN22 = \'{}\' or COLUMN23 = \'{}\' or COLUMN24 = \'{}\' or COLUMN25 = \'{}\' or COLUMN26 = \'{}\' or COLUMN27 = \'{}\' or COLUMN28 = \'{}\' or COLUMN29 = \'{}\' or COLUMN30 = \'{}\' or COLUMN31 = \'{}\' or COLUMN32 = \'{}\' or COLUMN33 = \'{}\' or COLUMN34 = \'{}\' or COLUMN35 = \'{}\' or COLUMN36 = \'{}\') and NOT EXISTS(select COLUMN1 from SYNONIMY2 where COLUMN1 = \'{}\') and rownum=1 '.format(
                word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word))
        for i in thesaurus:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile

In [12]:
def pdfparser(data, nr):
    file = io.open(data, mode="r", encoding='utf-8-sig')
    myfile = file.read()
    myfile = myfile.lower()
    myfile = clearTxt(myfile)
    myfile = myfile.split(' ')
    replaceFile = slowikPolskiego(myfile)
    replaceFile = stopList(replaceFile)
    replaceFile = slowikSynonimow3(replaceFile)
    replaceFile = stopList(replaceFile)
    txt = ''
    for i in range(0, len(replaceFile), 1):
        txt = txt + ' ' + (replaceFile.__getitem__(i))
        if i % 200 == 0 and i != 0:
            cur4.execute(
                'INSERT INTO "C##PPAWLUK"."DOKUMENT" (NAZWA_DOKUMENTU, TEKST) VALUES( \'{}\', \'{}\')'.format(
                    nameDocuments[nr], txt))
            cur4.execute('commit')
            txt = ''
    if (txt != ''):
        cur5.execute(
            'INSERT INTO "C##PPAWLUK"."DOKUMENT" (NAZWA_DOKUMENTU, TEKST) VALUES( \'{}\', \'{}\')'.format(
                nameDocuments[nr], txt))
        cur5.execute('commit')

In [13]:
for i in range(0, len(nameDocuments), 1):
    pdfparser("D:\BSpy\!materiały na studia\Praca inżynierska\Implementacja\PorownywanieDokumentow\dokumenty\{}".format(
        nameDocuments[i]), i)

dziennik
ustawa
rzeczpospolita
polski
warszawa
dzień
październik
rok
pozycja
wyrok
trybunał
konstytucyjny
z
październik
rok
akt
trybunał
konstytucyjny
w
przewodniczyć
tuleja
sprawozdawca
po
w
artykuł
usta
punkt
ustawa
z
listopad
rok
o
i
postępowanie
przed
konstytucyjny
dziennik
u
na
niejawny
w
październik
rok
skarga
konstytucyjny
o
zgodność
artykuł
ustawa
z
czerwiec
rok
kodeks
postępowanie
karny
dziennik
u
pozycja
z
w
w
wyłączać
dopuszczalność
zaskarżyć
zażalenie
postanowienie
wydać
na
artykuł
kodeks
postępowanie
karny
i
się
oskarżyciel
posiłkowy
o
mowa
w
kodeks
postępowanie
karny
z
w
z
usta
artykuł
usta
artykuł
usta
i
konstytucja
rzeczpospolita
polski
orzekać
artykuł
ustawa
z
czerwiec
rok
kodeks
postępowanie
karny
dziennik
u
pozycja
z
w
obowiązujący
do
rok
to jest
do
poprzedzać
wejście
w
artykuł
punkt
ustawa
z
wrzesień
rok
o
ustawa
kodeks
postępowanie
karny
oraz
inny
dziennik
u
w
wyłączać
dopuszczalność
zaskarżyć
zażalenie
postanowienie
wydać
na
artykuł
ustawa
kodeks
postępowanie
karn

centymetr
strona
główny
płat
sztandar
umieszczać
górny
częsty
godło
napis
służba
dolny
częsty
godło
napis
bóg
ojczyzna
alba
napis
bóg
honor
ojczyzna
napis
bóg
honor
ojczyzna
mieć
kształt
półkole
litera
wysokość
centymetr
wykonać
kolor
złoty
wzór
strona
główny
płat
sztandar
określić
załącznik
numer
rozporządzenie
strona
odwrotny
płat
sztandar
środek
płat
znajdować
wizerunek
święty
wysokość
centymetr
wizerunek
święty
umieszczać
pełny
nazwa
jednostka
pod
święty
nazwa
siedziba
jednostka
litera
wysokość
centymetr
wykonać
kolor
srebrny
dolny
róg
strona
odwrotny
płat
sztandar
umieszczać
lewa
róg
rok
powołanie
jednostka
prawa
róg
rok
nadanie
sztandar
wzór
strona
odwrotny
płat
sztandar
określić
załącznik
numer
rozporządzenie
strona
odwrotny
płat
sztandar
on
róg
móc
bycie
umieścić
herb
miejscowość
ziem
oznaka
pożarniczy
państwowy
straż
pożarny
emblemat
inicjał
fundator
sztandar
minister
sprawa
wewnętrzny
administracja
kierować
dziać
administracja
rządowy
sprawa
wewnętrzny
podstawa
usta
punkt
roz

czas
bar
d
bar
środek
i
oznakowanie
nawigacyjny
bar
światowy
morski
system
łączność
alarmowy
i
bar
nawigacyjny
mapa
morski
bar
publikacja
nautyczny
bar
kompas
bar
procedura
bezpieczeństwo
na
bar
zasada
bezpieczeństwo
praca
i
praktyk
morski
bar
lin
i
bar
holować
i
przyrząd
pomiarowy
bar
kotwiczyć
bar
cumować
bar
e
bar
podstawa
meteorologia
i
meteorologiczny
bar
wiatr
i
morski
bar
prognozować
pogoda
bar
dziennik
ustawa
przedmiot
podstawowy
a
obliczenie
kartograficzny
i
geodezja
fizyczny
system
współrzędna
pomiar
lądowy
niwelacja
odwzorować
kartograficzny
trygonometria
teoria
obserwacja
i
mały
kwadrat
bar
oceanografia
masa
wodny
i
cyrkulacja
geologia
i
geologia
geofizyk
morze
przedmiot
specjalistyczny
a
system
odniesienie
statek
i
pomiarowy
światowy
system
nawigacja
satelitarny
system
inercjalny
system
nawigacja
podwodny
pozycjonowanie
na
bar
hydroakustyczny
urządzenie
pomiarowy
podstawa
hydroakustyk
system
jednowiązkowy
system
sonarowy
system
rozpraszanie
wsteczny
wiązka
hydroakustyczny


siedziba
biuro
hydrograficzny
marynarka
wojenny
pisemny
potwierdzenie
fakt
odbiór
wzór
dyplom
hydrograf
morski
kategoria
być
załącznik
numer
rozporządzenie
bar
być
załącznik
numer
rozporządzenie
dziennik
ustawa
pozycja
weryfikacja
instytucja
organizacja
ośrodek
szkolenie
odbywać
rozpatrzenie
wniosek
akceptacja
program
szkolenie
złożony
przez
organizacja
alba
ośrodek
szkolenie
szef
rozpatrywać
wniosek
który
usta
termin
dzień
dzień
wpływ
informować
instytucja
organizacja
alba
ośrodek
szkolenie
uznanie
alba
odmowa
uznanie
wniosek
który
usta
zawierać
następować
informacja
nazwa
cel
szkolenie
ten
struktura
program
obejmować
charakterystyka
poszczególny
grupa
przedmiot
moduł
szkoleniowy
wraz
część
szkolenie
praktyczny
seminarium
ćwiczenia
terenowy
egzamin
czas
trwanie
poszczególny
moduł
przedmiot
całkowity
liczba
godzina
wykład
laboratorium
kierować
kierować
ćwiczenia
praktyczny
przewidywać
student
uczestnik
bar
szczegółowy
kart
przedmiot
wszyscy
przewidzieć
program
zawierać
cel
kształcenie


nr
wzór
dyplomu
hydrografa
morskiego
kategorii
dyplom
hydrografa
morskiego
numer
biuro
hydrograficzne
marynarki
wojennej
okaziciel
niniejszego
dyplomu
posiada
wiedzę
umiejętności
doświadczenie
wykonywaniu
prac
hydrograficznych
stosownie
wymagań
określonych
w
iho
standards
of
competence
for
category
hydrographic
surveyors
miejsce
na
kategoria
b
i
mp
podpis
posiadacza
dyplomu
szef
biura
hydrograficznego
marynarki
wojennej
gdynia
dnia
dziennik
ustawa
rzeczpospolita
polski
warszawa
dzień
październik
rok
pozycja
rozporządzenie
prezes
rad
minister
z
październik
rok
w
dokonanie
przeniesienie
niektóry
planować
wydatek
budżet
państwo
w
kwota
wynagrodzenie
określić
w
budżetowy
na
na
artykuł
ustawa
z
kwiecień
rok
o
ustawa
o
administracja
rządowy
oraz
o
informacja
przestrzenny
dziennik
u
zarządzać
się
następować
rozporządzenie
określać
przeniesienie
niektóry
planować
wydatek
budżet
państwo
w
kwota
wynagrodzenie
między
budżet
państwo
w
dotyczyć
ministerstwo
cyfryzacja
oraz
inwestycja
i
w
na
dziać
i

In [14]:
cur.close()
cur1.close()
cur2.close()
cur3.close()
cur4.close()
cur5.close()

In [15]:
connection.close()